In [1]:
import os
import csv
import re
from bs4 import BeautifulSoup

# Ordner mit gespeicherten HTML-Dateien
ordner_pfad = "../ALL DATA IMMOWELT"

# Ergebnisse speichern
results = []

# Mapping: Ortsteil dem zugehörigen Bezirk zuordnen
ORTSTEIL_ZU_BEZIRK = {
    # Mitte
    "Gesundbrunnen": "Mitte",
    "Hansaviertel": "Mitte",
    "Mitte": "Mitte",
    "Moabit": "Mitte",
    "Tiergarten": "Mitte",
    "Wedding": "Mitte",

    # Friedrichshain-Kreuzberg
    "Friedrichshain": "Friedrichshain-Kreuzberg",
    "Kreuzberg": "Friedrichshain-Kreuzberg",

    # Pankow
    "Blankenburg": "Pankow",
    "Blankenfelde": "Pankow",
    "Buch": "Pankow",
    "Französisch Buchholz": "Pankow",
    "Heinersdorf": "Pankow",
    "Karow": "Pankow",
    "Niederschönhausen": "Pankow",
    "Pankow": "Pankow",
    "Prenzlauer Berg": "Pankow",
    "Rosenthal": "Pankow",
    "Stadtrandsiedlung Malchow": "Pankow",
    "Weißensee": "Pankow",
    "Wilhelmsruh": "Pankow",

    # Charlottenburg-Wilmersdorf
    "Charlottenburg": "Charlottenburg-Wilmersdorf",
    "Charlottenburg-Nord": "Charlottenburg-Wilmersdorf",
    "Grunewald": "Charlottenburg-Wilmersdorf",
    "Halensee": "Charlottenburg-Wilmersdorf",
    "Schmargendorf": "Charlottenburg-Wilmersdorf",
    "Westend": "Charlottenburg-Wilmersdorf",
    "Wilmersdorf": "Charlottenburg-Wilmersdorf",

    # Spandau
    "Falkenhagener Feld": "Spandau",
    "Gatow": "Spandau",
    "Hakenfelde": "Spandau",
    "Haselhorst": "Spandau",
    "Kladow": "Spandau",
    "Siemensstadt": "Spandau",
    "Spandau": "Spandau",
    "Staaken": "Spandau",
    "Wilhelmstadt": "Spandau",

    # Steglitz-Zehlendorf
    "Dahlem": "Steglitz-Zehlendorf",
    "Lankwitz": "Steglitz-Zehlendorf",
    "Lichterfelde": "Steglitz-Zehlendorf",
    "Nikolassee": "Steglitz-Zehlendorf",
    "Schlachtensee": "Steglitz-Zehlendorf",
    "Steglitz": "Steglitz-Zehlendorf",
    "Wannsee": "Steglitz-Zehlendorf",
    "Zehlendorf": "Steglitz-Zehlendorf",

    # Tempelhof-Schöneberg
    "Friedenau": "Tempelhof-Schöneberg",
    "Lichtenrade": "Tempelhof-Schöneberg",
    "Mariendorf": "Tempelhof-Schöneberg",
    "Marienfelde": "Tempelhof-Schöneberg",
    "Schöneberg": "Tempelhof-Schöneberg",
    "Tempelhof": "Tempelhof-Schöneberg",

    # Neukölln
    "Britz": "Neukölln",
    "Buckow": "Neukölln",
    "Gropiusstadt": "Neukölln",
    "Neukölln": "Neukölln",
    "Rudow": "Neukölln",

    # Treptow-Köpenick
    "Adlershof": "Treptow-Köpenick",
    "Altglienicke": "Treptow-Köpenick",
    "Alt-Treptow": "Treptow-Köpenick",
    "Baumschulenweg": "Treptow-Köpenick",
    "Bohnsdorf": "Treptow-Köpenick",
    "Grünau": "Treptow-Köpenick",
    "Johannisthal": "Treptow-Köpenick",
    "Köpenick": "Treptow-Köpenick",
    "Müggelheim": "Treptow-Köpenick",
    "Niederschöneweide": "Treptow-Köpenick",
    "Oberschöneweide": "Treptow-Köpenick",
    "Plänterwald": "Treptow-Köpenick",
    "Rahnsdorf": "Treptow-Köpenick",
    "Schmöckwitz": "Treptow-Köpenick",
    "Friedrichshagen": "Treptow-Köpenick",

    # Marzahn-Hellersdorf
    "Biesdorf": "Marzahn-Hellersdorf",
    "Hellersdorf": "Marzahn-Hellersdorf",
    "Kaulsdorf": "Marzahn-Hellersdorf",
    "Mahlsdorf": "Marzahn-Hellersdorf",
    "Marzahn": "Marzahn-Hellersdorf",

    # Lichtenberg
    "Alt-Hohenschönhausen": "Lichtenberg",
    "Falkenberg": "Lichtenberg",
    "Fennpfuhl": "Lichtenberg",
    "Friedrichsfelde": "Lichtenberg",
    "Karlshorst": "Lichtenberg",
    "Lichtenberg": "Lichtenberg",
    "Malchow": "Lichtenberg",
    "Neu-Hohenschönhausen": "Lichtenberg",
    "Rummelsburg": "Lichtenberg",
    "Wartenberg": "Lichtenberg",

    # Reinickendorf
    "Borsigwalde": "Reinickendorf",
    "Frohnau": "Reinickendorf",
    "Heiligensee": "Reinickendorf",
    "Hermsdorf": "Reinickendorf",
    "Konradshöhe": "Reinickendorf",
    "Lübars": "Reinickendorf",
    "Märkisches Viertel": "Reinickendorf",
    "Reinickendorf": "Reinickendorf",
    "Tegel": "Reinickendorf",
    "Waidmannslust": "Reinickendorf",
    "Wittenau": "Reinickendorf",
}

# Funktionen zur Ortsteil- und Bezirksermittlung
def extrahiere_ortsteil(adresse: str) -> str:
    teile = [t.strip() for t in adresse.split(",")]

    if len(teile) >= 2:
        # Letztes Element ist Berlin(PLZ), Ortsteil davor
        ortsteil = teile[-2]
        return ortsteil
    return "Nicht erkannt"


def bestimme_bezirk(ortsteil: str) -> str:
    return ORTSTEIL_ZU_BEZIRK.get(ortsteil, "Unbekannt")

def parse_zahl(wert):
    try:
        # Entfernt geschützte Leerzeichen, €, Punkte etc., ersetzt Kommas durch Punkte
        zahl = re.sub(r"[^\d,]", "", wert)        # Nur Ziffern und Komma behalten
        zahl = zahl.replace(",", ".")             # Komma → Punkt
        return float(zahl)
    except:
        return None



# Alle HTML-Dateien im Ordner durchgehen zum scrappen
for dateiname in os.listdir(ordner_pfad):
    if dateiname.endswith(".html"):
        dateipfad = os.path.join(ordner_pfad, dateiname)
        print(f"Verarbeite Datei: {dateipfad}")

        with open(dateipfad, "r", encoding="utf-8") as f:
            html = f.read()

        soup = BeautifulSoup(html, "html.parser")

        # URL über Canonical-Tag oder OG-Tags extrahieren
        url = "Nicht gefunden"
        
        # Versuch 1: canonical link
        canonical = soup.find("link", rel="canonical")
        if canonical and canonical.get("href"):
            url = canonical["href"]
        
        # Versuch 2: og:url Meta-Tag
        if url == "Nicht gefunden":
            meta_og = soup.find("meta", property="og:url")
            if meta_og and meta_og.get("content"):
                url = meta_og["content"]


        # Titel
        titel_el = soup.select_one('h2[data-testid="aviv.CDP.Sections.Description.MainDescription.Title"]')
        titel = titel_el.get_text(strip=True) if titel_el else "Nicht gefunden"

        # Beschreibung
        beschreibung_el = soup.select_one('div[data-testid="aviv.CDP.Sections.Description.MainDescription.Expandable-text"]')
        beschreibung = beschreibung_el.get_text(strip=True) if beschreibung_el else "Nicht gefunden"

        beschreibung_gesamt = f"{titel}\n\n{beschreibung}" if titel != "Nicht gefunden" else beschreibung

        # Preis
        preis_el = soup.select_one('[data-testid="aviv.CDP.Sections.Hardfacts.Price.Value"]')
        preis = preis_el.get_text(strip=True) if preis_el else "Nicht gefunden"

        # Wohnfläche und Zimmer
        wohnflaeche, zimmer = "Nicht gefunden", "Nicht gefunden"
        for fact in soup.select(".css-j7qwjs"):
            text = fact.get_text(strip=True)
            if "m²" in text:
                wohnflaeche = text
            if "Zimmer" in text:
                zimmer = text

        # Preis pro qm
        preis_num = parse_zahl(preis)
        wohnflaeche_num = parse_zahl(wohnflaeche)
        
        if preis_num and wohnflaeche_num and wohnflaeche_num > 0:
            preis_pro_qm = round(preis_num / wohnflaeche_num, 2)
        else:
            preis_pro_qm = "Nicht berechenbar"

        # Etage
        etage = "Keine Angabe"
        for fact in soup.select(".css-1c3h18e"):
            text = fact.get_text(strip=True)
            if "Geschoss" in text:
                etage = text

        # Adresse extrahieren
        adresse_el = soup.select_one('button[data-testid="cdp-location-address"] > div.css-1ytyjyb')
        adresse = adresse_el.get_text(strip=True) if adresse_el else "Keine Angabe"

        # Bezirk
        ortsteil = extrahiere_ortsteil(adresse)
        bezirk = bestimme_bezirk(ortsteil)



        # Energieeffizienzklasse extrahieren
        energie_el = soup.find("div", attrs={"data-testid": "aviv.CDP.Sections.Energy.Preview.EfficiencyClass"})
        energie = energie_el.get_text(strip=True) if energie_el else "Keine Angabe"
        
        # Baujahr extrahieren
        baujahr_el = soup.find("span", attrs={"data-testid": "aviv.CDP.Sections.Energy.Features.yearOfConstruction"})
        baujahr = baujahr_el.get_text(strip=True) if baujahr_el else "Keine Angabe"

        
        # Volltextsuche für bestimmte Features
        html_lower = html.lower()
        
        balkon = "Ja" if any(w in html_lower for w in ["balkon", "terrasse", "garten"]) else "Nein"
        keller = "Ja" if "keller" in html_lower else "Nein"
        aufzug = "Ja" if "aufzug" in html_lower else "Nein"
        stellplatz = "Ja" if any(w in html_lower for w in ["stellplatz", "garage"]) else "Nein"
        
        # Prüfen, ob in der Beschreibung etwas über Möbel steht
        moebel_schlagworte = ["möbliert", "möbel", "einrichtung", "vollmöbliert", "teilmöbliert"]
        beschreibung_lower = beschreibung_gesamt.lower()
        moebliert = "Ja" if any(wort in beschreibung_lower for wort in moebel_schlagworte) else "Nein"
        
        # Ausstattungspunktzahl berechnen
        ausstattungsgrad = sum([
            moebliert == "Ja",
            balkon == "Ja",
            keller == "Ja",
            aufzug == "Ja",
            stellplatz == "Ja"
        ])
        
        results.append({
            "Datei": dateiname,
            "URL": url,
            "Beschreibung": beschreibung_gesamt,
            "Preis": preis,
            "Preis pro qm in euro": preis_pro_qm,
            "Wohnfläche": wohnflaeche,
            "Zimmer": zimmer,
            "Etage": etage,
            "Adresse": adresse,
            "Bezirk": bezirk,
            "Möbeliert": moebliert,
            "Balkon/Terrasse/Garten": balkon,
            "Keller": keller,
            "Aufzug": aufzug,
            "Stellplatz": stellplatz,
            "Energieeffizienzklasse": energie,
            "Baujahr": baujahr,
            "Ausstattungsgrad": ausstattungsgrad
        })



# CSV speichern in 'data'-Ordner
if results:
    # Neuen Ordner erstellen, falls nicht vorhanden
    os.makedirs("../dat", exist_ok=True)

    # Zielpfad für die CSV
    csv_pfad = os.path.join("../dat", "immowelt_wohnungen_complete.csv")

    with open(csv_pfad, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=results[0].keys())
        writer.writeheader()
        writer.writerows(results)
    print(f"Fertig! Daten gespeichert in '{csv_pfad}'")
else:
    print("Keine Daten gefunden.")


Verarbeite Datei: ALL DATA IMMOWELT/Wohnung 47 m² 610 € zur Miete Mariendorf,Berlin (12107).html
Verarbeite Datei: ALL DATA IMMOWELT/Wohnung 111 m² 2050 € zur Miete Elsa-Neumann-Straße 53,Siemensstadt,Berlin (13629).html
Verarbeite Datei: ALL DATA IMMOWELT/Wohnung 62 m² 1600 € zur Miete Bellevuestraße 1,Tiergarten,Berlin (10785).html
Verarbeite Datei: ALL DATA IMMOWELT/Wohnung 80 m² 880 € zur Miete Schmargendorf,Berlin (14193).html
Verarbeite Datei: ALL DATA IMMOWELT/Wohnung 64 m² 1750 € zur Miete Else-Lasker-Schüler-Straße 4,Schöneberg,Berlin (10783).html
Verarbeite Datei: ALL DATA IMMOWELT/Wohnung 61 m² 661 € zur Miete Tegel,Berlin (13509).html
Verarbeite Datei: ALL DATA IMMOWELT/Wohnung 67 m² 700 € zur Miete Wilmersdorf,Berlin (10777).html
Verarbeite Datei: ALL DATA IMMOWELT/Wohnung 49 m² 450 € zur Miete Kreuzberg,Berlin (10969).html
Verarbeite Datei: ALL DATA IMMOWELT/Wohnung 41 m² 266 € zur Miete Friedrichshain,Berlin (10245).html
Verarbeite Datei: ALL DATA IMMOWELT/Wohnung 50 m